## System certificates

This section describes the system certificate roots. These are the certificates available to the system and system libraries. 

_I will be showing the debian versions and adding in some of the Red Hat version commands, in some cases however it is more complicated to show a relevant analog between the 2 implementations and will not have a Red Hat version._ 

_An additional note: we are not considering the implications of flatpaks, snaps, or other container systems, in those cases the certificate trust may be embedded and use different structures. However, much of the same rules and structure apply in those container implmentations, so this information should still be relevant._

### The ca-certificates package

Taking a look into the package using `dpkg-query` command

```
# rpm version
rpm -q ca-certificates --info
```

In [ ]:
%%bash
dpkg-query -s ca-certificates

Let us take a look at the output of the files that are loaded from the package. 

As an example, we are interested in looking at the Let's Encrypt root certificate that we can use in later examples. 

Using the `dpkg-query` command we can pull out all the files provided by the package. In this command I am `grep`ing out the ISRG_Root certificate along with 2 before and 2 after to show a part of the output we are interested in. 

Depending on the implementation the ca-certificates package may use mulitple files, as is the case in debian, or it can use a single bundle file, as is the case in Red Hat. This example will not work the same in Red Hat since it bundles the root certificates into a single file, but we can still find the proper Let's encrypt certificate.

```
# rpm version of command
rpm -q ca-certifcates --dump | ca-bundle.trust.crt 
# should show ca-bundle.trust.crt in the /etc/pki/ca-trust/extracted/openssl folder
# grep of the "#" will parse out the certificate labels
grep "#" /etc/pki/ca-trust/extracted/openssl/ca-bundle.trust.crt | grep -A2 -B2 DST
```

In [ ]:
%%bash
dpkg-query -S ca-certificates | grep -A2 -B2 DST_Root_CA_X3

To confirm the location of the certificates we can look in the directory. 

In [ ]:
%%bash
ls -l /usr/share/ca-certificates/mozilla/DST_Root_CA_X3.crt

The `ca-certificates` package also provides a utility to update the system trust roots using a tool called `update-ca-certificates`. This utility unifies the certificate locations used by the system.

In the case of the package management, the utility has already been executed, we will later look at adding certificates to the system.

For the current example we can look into the system certificate location to see that the utility has created a link to the location from the package as well as a hash link. 

_The hash link is an operation provided by some implementations of openssl called `rehash`. The hash links are used by some applications, such as some options for the `pam_pkcs11` implementation, but I have found it unclear as to why and when it is needed. If the `rehash` openssl option is not available, some systems have a script for `c_rehash` that peforms the same action over a directory. The operation can also be done manually per file if needed. See [man page](https://www.openssl.org/docs/man1.1.0/man1/rehash.html) for more detail._ 

_In the Red Hat package the system operates differently, the use of a utility `update-ca-trust` generates the ca-trust.bundle.crt as well as multiple other links and operations associated to the `p11-kit` package which we will look at later. For the purposes of this document we will not dig as deep into the Red Hat utility._

In [ ]:
%%bash
ls -l /etc/ssl/certs | grep DST_Root_CA_X3

Taking a look inside of the certificate. 

In [ ]:
%%bash
cat /etc/ssl/certs/DST_Root_CA_X3.pem

Using the `openssl` utility, which we will go into greater detail in following sections, we can parse out the subject of the certificate. 

In [ ]:
%%bash
openssl x509 -in /etc/ssl/certs/DST_Root_CA_X3.pem -noout -subject

As a simple example we can now look at the ohiolinux.org ssl certificate and its association to the DST_Root_CA_X3 certificate.

In [ ]:
%%bash
openssl s_client -4 -quiet -no_ign_eof ohiolinux.org:443 <<< "Q"

## Missing system certificates

Let us take a look at the behavior of the system if we are missing a root certificate.

In [ ]:
%%bash
sudo rm -f /etc/ssl/certs/*

In [ ]:
%%bash
ls -l /etc/ssl/certs/*

Now when we attempt to look at the ohiolinunx.org ssl certificate we can see an error with inability to get local issuer certificate.

In [ ]:
%%bash
openssl s_client -4 -quiet -no_ign_eof ohiolinux.org:443 <<< "Q"

We can see that we are unable to get the issuer certificate now.

## Adding system certificates

In order to add certificates back into the system we can now use the `update-ca-certificates` utility. 

We will use this utility later when discussing more advanced uses of the certificate system, so for now we can look at the base package to restore our certificates. The base certificates are identified in a configuration file, the configuration file allows udpates to the certificate trust. 

_In a Red Hat system this configuration file does not exist, instead it uses a file structure inside the /etc/pki/ca-trust folder to accomplish the same function._

In [ ]:
%%bash
head -n 11 /etc/ca-certificates.conf

Since we know that the ca-certificates package does not place the certificates into the /etc/ssl/certs directory, we can now refresh the certificates back by executing the utility.

In [ ]:
%%bash
sudo update-ca-certificates

And to look at the Let's Encrypt root cert again.

In [ ]:
%%bash
ls -l /etc/ssl/certs | grep DST_Root_CA_X3.pem 

We can try again to see if we have the issuer certificate.

In [ ]:
%%bash
openssl s_client -4 -quiet -no_ign_eof ohiolinux.org:443 <<< "Q"

Here the verify returns 1 and we can see the issuer certificate subject.

### Some final words

In this section we have looked at the ca-certificates pacakge and its implementation into a Linux system.

In later sections we will touch on how this ca-certificates structure is used in certificate verification and validation.